In [1]:
#%pip install ipywidgets

### MAIN CODE LOOP

In [2]:
import os
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from scipy import stats

# normalize the Hr data
from sklearn.preprocessing import MinMaxScaler



In [3]:
# create a directory to store the generated files
try:
    os.makedirs("../generated_data/train/", exist_ok=True)
    os.makedirs("../generated_data/test/", exist_ok=True)
    
    os.makedirs("../generated_data/train/Hr", exist_ok=True)
    os.makedirs("../generated_data/test/Hr", exist_ok=True)
    
    os.makedirs("../generated_data/train/HaHb", exist_ok=True)
    os.makedirs("../generated_data/test/HaHb", exist_ok=True)
    print("Directory created successfully or already exists.")
except Exception as e:
    print(f"Error creating directory: {e}")

Directory created successfully or already exists.


In [4]:
def get_HaHb_array(data,hydropathy_scale):
    HaHb_list = []
    #print(len(HaHb_list))
    # For one scale get all HaHb values
    # iterate over the dataset
    for i in range(data.shape[0]):
        row = data.iloc[i]
        resA_ = row['resA']
        resB_ = row['resB']
        resB_1 = row['resB_1']
        resB_2 = row['resB_2']
        resB_3 = row['resB_3']
        resB_4 = row['resB_4']
        resB_5 = row['resB_5']
        resB_6 = row['resB_6']
        resB_7 = row['resB_7']
        resB_8 = row['resB_8']
        resB_9 = row['resB_9']

        # get values from the scale
        resA_scale_value = hydropathy_scale[resA_].iloc[0]
        resB_scale_value = hydropathy_scale[resB_].iloc[0]
        resB_1_scale_value = hydropathy_scale[resB_1].iloc[0]
        resB_2_scale_value = hydropathy_scale[resB_2].iloc[0]
        resB_3_scale_value = hydropathy_scale[resB_3].iloc[0]
        resB_4_scale_value = hydropathy_scale[resB_4].iloc[0]
        resB_5_scale_value = hydropathy_scale[resB_5].iloc[0]
        resB_6_scale_value = hydropathy_scale[resB_6].iloc[0]
        resB_7_scale_value = hydropathy_scale[resB_7].iloc[0]
        resB_8_scale_value = hydropathy_scale[resB_8].iloc[0]
        resB_9_scale_value = hydropathy_scale[resB_9].iloc[0]

        # calculate HaHb values
        currentcombination = [resA_scale_value*resB_scale_value, 
        resA_scale_value*resB_1_scale_value, 
        resA_scale_value*resB_2_scale_value, 
        resA_scale_value*resB_3_scale_value, 
        resA_scale_value*resB_4_scale_value, 
        resA_scale_value*resB_5_scale_value, 
        resA_scale_value*resB_6_scale_value, 
        resA_scale_value*resB_7_scale_value, 
        resA_scale_value*resB_8_scale_value, 
        resA_scale_value*resB_9_scale_value]

        HaHb_list.append(currentcombination)
    
    data = np.array(HaHb_list)
    print(data.shape)
    # Find the minimum value in the array
    min_value = np.min(data)

    # Shift the array values to be all positive
    shifted_data = data - min_value
    print(shifted_data.shape)

    return shifted_data

In [5]:
def get_values_for_abc(HaHb_min,HaHb_max,HaHb_median):
    x1 = HaHb_min
    x2 = HaHb_max
    x3 = HaHb_median
    A = np.array([
        [-x1**2, x1, 1],
        [-x2**2, x2, 1],
        [-x3**2, x3, 1]
    ])
    
    # Constants matrix
    B = np.array([0, 0, 1])
    
    # Solving the system of equations
    solution = np.linalg.solve(A, B)
    a, b, c = solution
    # print(a,b,c)
    return a,b,c

In [6]:
"""def get_values_for_abc(HaHb_max,HaHb_min,HaHb_median):
    x1 = HaHb_min
    x2 = HaHb_max
    x3 = HaHb_median

    a = -1 / (x1 * x2 - x1 * x3 - x2 * x3 + x3**2)
    b = (- (x1 + x2)) / (x1 * x2 - x1 * x3 - x2 * x3 + x3**2)
    c = (x1 * x2) / (x1 * x2 - x1 * x3 - x2 * x3 + x3**2)
    return a,b,c"""

'def get_values_for_abc(HaHb_max,HaHb_min,HaHb_median):\n    x1 = HaHb_min\n    x2 = HaHb_max\n    x3 = HaHb_median\n\n    a = -1 / (x1 * x2 - x1 * x3 - x2 * x3 + x3**2)\n    b = (- (x1 + x2)) / (x1 * x2 - x1 * x3 - x2 * x3 + x3**2)\n    c = (x1 * x2) / (x1 * x2 - x1 * x3 - x2 * x3 + x3**2)\n    return a,b,c'

In [7]:
# read the main dataset
filename = "../dataset/train/dataset.txt"
dataset = pd.read_csv(filename)

# drop the pdb column
data = dataset.drop(columns=['pdb'], axis=1)

In [8]:
# read the test dataset
# read the main dataset
filename = "../dataset/test/dataset.txt"
dataset_test = pd.read_csv(filename)

# drop the pdb column
data_test = dataset_test.drop(columns=['pdb'], axis=1)
print(data_test.shape)
data_test.head()

(1282, 11)


,resA,resB,resB_1,resB_2,resB_3,resB_4,resB_5,resB_6,resB_7,resB_8,resB_9
0,HIS,HIS,THR,GLN,ASP,LEU,ILE,PRO,TYR,VAL,ARG
1,GLU,GLY,PRO,SER,GLU,PHE,LEU,TYR,THR,GLU,ARG
2,PHE,VAL,ILE,SER,LEU,VAL,ASP,GLU,PHE,ILE,GLU
3,GLU,LYS,LEU,LEU,ALA,ARG,GLU,GLU,LEU,ASP,ASP
4,GLY,THR,GLU,ARG,PHE,THR,ARG,GLN,PHE,SER,LEU


In [9]:
data_combined = pd.concat([data, data_test], axis=0)
print(data_combined.shape)
data_combined.head()

(4272, 11)


,resA,resB,resB_1,resB_2,resB_3,resB_4,resB_5,resB_6,resB_7,resB_8,resB_9
0,TRP,LEU,LEU,ILE,TRP,MET,ILE,TYR,LYS,ILE,TRP
1,THR,HIS,GLY,GLY,VAL,THR,GLY,VAL,ILE,THR,ASP
2,LYS,HIS,ASP,THR,GLU,LYS,ARG,TYR,GLU,LEU,ASN
3,LYS,HIS,PHE,CYS,PHE,THR,CYS,TRP,GLU,GLY,ASN
4,VAL,LEU,ALA,LYS,LEU,VAL,GLU,LEU,ASP,SER,SER


In [10]:
# loop through hydropathy folder and pick scale one by one, generate HaHb values and save them in a list
# iterate over the directory
directory = "../hydropathy/"

# Initialize the MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

for file_path in tqdm(os.listdir(directory)):

    # ----------------------------------------------------------------------------
    # calculate HaHb
    # final list to save the HaHb values
    HaHb_list = []

    hydropathy_scale = pd.read_csv(directory + file_path,header = None)
    hydropathy_scale.columns = ['aa', 'hydropathy']

    # typecast hydropathy values to float
    hydropathy_scale['hydropathy'] = hydropathy_scale['hydropathy'].astype(float)

    hydropathy_scale = hydropathy_scale.T
    # make first row as header
    new_header = hydropathy_scale.iloc[0]
    hydropathy_scale = hydropathy_scale[1:]
    hydropathy_scale.columns = new_header
    hydropathy_scale.reset_index(drop=True, inplace=True)
    #print("train")
    # for training data
    HaHb_train = get_HaHb_array(data,hydropathy_scale)

    #print("test")
    # for test data
    HaHb_test = get_HaHb_array(data_test,hydropathy_scale)
    
    #print("combined")
    # for a,b,c calculation
    HaHb_combined = get_HaHb_array(data_combined,hydropathy_scale)
    #print(HaHb.shape)
    
    # apply log transformation on HaHb_combined
    #HaHb_combined = np.log(HaHb_combined)

    # get mid,min and max values of HaHb
    HaHb_min = np.min(HaHb_combined)
    HaHb_max = np.max(HaHb_combined)
    
    # --------------------------- NEW UPDATE -------------------------------------
    # consider data from 5 to 75 percentile
    HaHb_filtered = HaHb_combined[(HaHb_combined >= np.percentile(HaHb_combined, 5)) & (HaHb_combined <= np.percentile(HaHb_combined, 75))]
    HaHb_median = np.median(HaHb_filtered)
    #print(HaHb_median)

    # ----------------------------------------------------------------------------
    #print(HaHb.shape)
    # calculate Hr
    try:
        # pre normalization
        a, b, c = get_values_for_abc(HaHb_max, HaHb_min, HaHb_median)
        
        # saving value to print later
        a_, b_, c_ = get_values_for_abc(HaHb_max, HaHb_min, HaHb_median)

        # save file_path,a_,b_,c_ values in a text file
        with open("a_b_c.txt", "a") as f:
            f.write(f"{file_path},{a_},{b_},{c_}\n")
        
        
        # post normalization
        #a,b,c = round(a/4,3),round(b/4,3),round(c/4,3)
        #print("after normalizaion",a,b,c)

    except np.linalg.LinAlgError as e:
        print(f"Processing file: {directory+file_path}")
        print(f"Error calculating values for a, b, c: {e}")
        print("Skipping this iteration due to singular matrix.")
        #print(HaHb)
        continue
    
    # get Hr values
    Hr = np.array(list(map(lambda HaHb_train: (-a * (HaHb_train ** 2)) + ((b * HaHb_train) + c), HaHb_train.flatten()))).reshape(HaHb_train.shape)
    Hr_test = np.array(list(map(lambda HaHb_test: (-a * (HaHb_test ** 2)) + ((b * HaHb_test) + c), HaHb_test.flatten()))).reshape(HaHb_test.shape)

    # normalize the Hr values
    Hr_train_norm = scaler.fit_transform(Hr)
    Hr_test_norm = scaler.fit_transform(Hr_test)

    #if Hr_train_norm has anu -ve values print file_path
    if np.any(HaHb_train < 0):
        print(f"Negative values found in Hr_train_norm for file: {file_path}")
        #print(Hr_train_norm
    if np.any(HaHb_test < 0):
        print(f"Negative values found in Hr_test_norm for file: {file_path}")
        #print(Hr_test_norm)

    # save train hr files
    file_name = file_path.replace('.csv', '')
    np.savetxt(f"../generated_data/train/HaHb/{file_name}.txt", HaHb_train)
    np.savetxt(f"../generated_data/train/Hr/{file_name}.txt", Hr_train_norm)
    # save test hr files
    np.savetxt(f"../generated_data/test/HaHb/{file_name}.txt", HaHb_test)
    np.savetxt(f"../generated_data/test/Hr/{file_name}.txt", Hr_test_norm)
    
    #print("________________________________")

  0%|          | 0/28 [00:00<?, ?it/s]

(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
Processing file: ../hydropathy/GOLD730101.csv
Error calculating values for a, b, c: Singular matrix
Skipping this iteration due to singular matrix.
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272, 10)
(2990, 10)
(2990, 10)
(1282, 10)
(1282, 10)
(4272, 10)
(4272

# Additional visualization and testing code

In [ ]:
# read Hr_L_hydrophobicity_scale.txt file
Hr = np.loadtxt("../generated_data/train/Hr/L_hydrophobicity_scale.txt")

# read /Users/pulkit/Desktop/mini_project/dsmp-2024-groupt3/dataset/train/hr.txt file
hr = np.loadtxt("../dataset/train/hr.txt")

print("For generated and original Hr")
print("generated",np.min(Hr), np.max(Hr), np.median(Hr))
print("original",np.min(hr), np.max(hr), np.median(hr))    

print(Hr.shape)
print()
print(hr.shape)

print(Hr)
print()
print(hr)

In [ ]:
import numpy as np

# Function to compute H_A * H_B given H_r, a, and b
def compute_HA_HB(H_r_values, a, b):
    H_A_H_B_values = []
    
    for H_r in H_r_values:
        # Compute the discriminant
        discriminant = b**2 - 4 * a * H_r

        if discriminant < 0:
            # No real solution; return NaN or handle differently
            H_A_H_B_values.append(np.nan)
        else:
            # Compute the two possible solutions
            root1 = (b + np.sqrt(discriminant)) / (2 * a)
            root2 = (b - np.sqrt(discriminant)) / (2 * a)
            
            # Choose the valid root (positive or within a reasonable range)
            H_A_H_B = root1 if root1 >= 0 else root2
            H_A_H_B_values.append(H_A_H_B)
    
    return np.array(H_A_H_B_values)

# Example dataset: Given H_r values, a and b from one hydropathy scale
# min max and median values of Hr
H_r_example = [np.min(hr), np.max(hr), np.median(hr)]  # Example H_r values
a_example = 0.033  # Example a value from a hydropathy scale
b_example = 0.363  # Example b value from a hydropathy scale

# Compute H_A * H_B values
H_A_H_B_computed = compute_HA_HB(H_r_example, a_example, b_example)
H_A_H_B_computed


In [ ]:
import matplotlib.pyplot as plt

# read Hr_L_hydrophobicity_scale.txt file original
Hr_generated = np.loadtxt("../generated_data/train/Hr/L_hydrophobicity_scale.txt")

# read Hr_L_hydrophobicity_scale.txt file original
Hr_original =np.loadtxt("../dataset/train/hr.txt")

# HaHb_L_hydrophobicity_scale.txt file original
HaHb = np.loadtxt("../generated_data/train/HaHb/L_hydrophobicity_scale.txt")

plt.figure(figsize=(12, 6))

# Scatter plot for Generated Hr vs HaHb
plt.subplot(1, 2, 1)
plt.scatter(Hr_generated.flatten(), HaHb.flatten(), label='Generated Hr vs HaHb', alpha=0.5)
plt.title("Generated Hr vs HaHb")
plt.xlabel("Hr (Generated)")
plt.ylabel("HaHb")
plt.legend()

# Scatter plot for Original Hr vs HaHb
plt.subplot(1, 2, 2)
plt.scatter(Hr_original.flatten(), HaHb.flatten(), label='Original Hr vs HaHb', alpha=0.5)
plt.title("Original Hr vs HaHb")
plt.xlabel("Hr (Original)")
plt.ylabel("HaHb")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize the array
Hr_generated_normalized = scaler.fit_transform(Hr_generated)


plt.scatter(Hr_generated_normalized.flatten(), HaHb.flatten(), label='Generated Hr vs HaHb', alpha=0.5)
plt.title("Generated Hr vs HaHb")
plt.xlabel("Hr (Generated)")
plt.ylabel("HaHb")
plt.legend()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Normalize the Hr array
Hr_generated_normalized = scaler.fit_transform(Hr_generated)

plt.figure(figsize=(12, 6))

# Scatter plot for Generated Hr vs HaHb
plt.subplot(1, 2, 1)
plt.scatter(Hr_generated_normalized.flatten(), HaHb.flatten(), label='Generated Hr vs HaHb', alpha=0.5)
plt.title("Generated Hr vs HaHb")
plt.xlabel("Hr (Generated)")
plt.ylabel("HaHb")
plt.legend()

# Scatter plot for Original Hr vs HaHb
plt.subplot(1, 2, 2)
plt.scatter(Hr_original.flatten(), HaHb.flatten(), label='Original Hr vs HaHb', alpha=0.5)
plt.title("Original Hr vs HaHb")
plt.xlabel("Hr (Original)")
plt.ylabel("HaHb")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
print(Hr_generated_normalized[0])
print("--------------------------------")
print()
print(Hr_original[0])

In [ ]:
from sklearn.preprocessing import RobustScaler
import numpy as np
 
# Example data
data = Hr_generated
 
# Robust Scaling
scaler = RobustScaler()
robust_scaled = scaler.fit_transform(data)


plt.figure(figsize=(12, 6))

# Scatter plot for Generated Hr vs HaHb
plt.subplot(1, 2, 1)
plt.scatter(robust_scaled.flatten(), HaHb.flatten(), label='Generated Hr vs HaHb', alpha=0.5)
plt.title("Generated Hr vs HaHb")
plt.xlabel("Hr (Generated)")
plt.ylabel("HaHb")
plt.legend()

# Scatter plot for Original Hr vs HaHb
plt.subplot(1, 2, 2)
plt.scatter(Hr_original.flatten(), HaHb.flatten(), label='Original Hr vs HaHb', alpha=0.5)
plt.title("Original Hr vs HaHb")
plt.xlabel("Hr (Original)")
plt.ylabel("HaHb")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
Hr_generated_normalized[0][1] - Hr_original[0][1]

In [ ]:
Hr[0][0] - hr[0][0]

In [ ]:
a_original = 0.033
b_original = 0.363
a_calculated = 0.132 
b_calculated = 1.456

a_calculated/a_original, b_calculated/b_original

In [ ]:
# Example dataset: Given H_r values, a and b from one hydropathy scale
# min max and median values of Hr
H_r_example = [np.min(Hr), np.max(Hr), np.median(Hr)]  # Example H_r values
a_example = 0.033  # Example a value from a hydropathy scale
b_example = 0.363  # Example b value from a hydropathy scale

# Compute H_A * H_B values
H_A_H_B_computed = compute_HA_HB(H_r_example, a_example, b_example)
H_A_H_B_computed


In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

# PCA is to find the direction (primary component) of the largest variance in the data, 
# and use projections in these directions to represent the data, 
# thereby reducing redundant information and retaining the main features of the data as much as possible in the lower dimensional space.

# set orders
amino_acids = ['ALA', 'CYS', 'ASP', 'GLU', 'PHE', 'GLY', 'HIS', 'ILE', 'LYS', 'LEU',
               'MET', 'ASN', 'PRO', 'GLN', 'ARG', 'SER', 'THR', 'VAL', 'TRP', 'TYR']

# Read all 28 hydrophilic scales
hydropathy_scales = []

scale_files = os.listdir("../hydropathy")
for scale_file in scale_files:
    df = pd.read_csv(f"../hydropathy/{scale_file}", delimiter=",", header=None, index_col=0)
    df = df.loc[amino_acids]  # Arrange amino acids in a unified order
    hydropathy_scales.append(df[1].values)  # get value 

# Convert all scale data into a matrix（形状：20×28，即 20 种氨基酸 × 28 个尺度）
scale_matrix = np.array(hydropathy_scales).T  # 转置使得行为氨基酸，列为不同尺度

#print("亲水性尺度矩阵形状：", scale_matrix.shape)  # (20, 28)



from sklearn.preprocessing import StandardScaler

# 1. Standardized data
scaler = StandardScaler()
scale_matrix_std = scaler.fit_transform(scale_matrix)

# 2. PCA dimensionality reduction (only take the first principal component)
pca = PCA(n_components=1)
principal_component = pca.fit_transform(scale_matrix_std)  # Calculate principal components

# 3. generate new scale
new_hydropathy_scale = principal_component.flatten()  # Flattening into 1D array
print("New Sclae file：", new_hydropathy_scale)

New Sclae file： [ 1.41824591  5.24243582 -5.11380416 -5.89382492  6.02764359 -0.72872686
 -2.27717033  6.34590405 -5.66471605  5.0928903   3.46664162 -4.23048941
 -0.4819167  -4.69258733 -7.41894613 -2.29324688 -0.76510997  4.92625187
  4.89540484  2.14512074]
